In [1]:
# !pip install pyproj
# !pip install shapely


## Imports

In [2]:
# Standaard bibliotheken
import json
import math

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import pandas as pd  # Handig voor lokale manipulatie na filtering

from pyproj import Transformer
from shapely.geometry import shape, Point
from shapely.strtree import STRtree  # VOOR OPTIMALISATIE: Ruimtelijke index

import folium
from folium.plugins import FeatureGroupSubGroup
import findspark

from ipywidgets import interact, IntSlider, Layout

## Spark initialisatie

Start een lokale SparkContext en SparkSession (nodig voor het verwerken van grote CSV-bestanden)

In [3]:
findspark.init()

# Start Spark Session
# We voegen 'spark.driver.memory' toe om zeker te zijn dat we genoeg geheugen hebben voor visualisaties
spark = SparkSession.builder \
    .appName("Verkeersanalyse Antwerpen") \
    .config("spark.driver.memory", "4g") \
    .getOrCreate()

## Coördinaattransformaties

Doordat de dataset van de overheid lokale coordinaten gebruikt en geen wereld long lat coordinaten gebruikt maken we een functie aan die deze kan omvormen

In [4]:

transformer = Transformer.from_crs("EPSG:31370", "EPSG:4326", always_xy=True)
transformer2 = Transformer.from_crs("EPSG:3857", "EPSG:4326", always_xy=True)

# Helper functie: zet x,y om naar lat,lon
def lambert_to_latlon(transformer, x, y):
    lon, lat = transformer.transform(x, y)
    return lat, lon

## Inladen ongevallen (CSV)

het bestand gebruikt '|' als scheidingsteken en heeft velden met aanhalingstekens.

In [5]:
ongevallen_df = spark.read.option("header", "true") \
    .option("delimiter", "|") \
    .option("multiLine", "true") \
    .option("quote", '"') \
    .option("escape", '"') \
    .csv("./data/OPENDATA_MAP_2017-2024.txt")


## Inladen basisscholen

Basisscholen dataset bevat coördinaten in X,Y kolommen als strings met komma als decimale scheiding.

In [6]:
basisonderwijs_df = spark.read.option("header", "true") \
                              .option("multiLine", "true") \
                              .option("quote", '"') \
                              .option("escape", '"') \
                              .csv('./data/basisonderwijs.csv')

# basisonderwijs_df.take(5)

## GeoJSON zones laden

Zone 30 polygons gebruiken we om na te gaan of een ongeval binnen een 30 km/u-zone valt.

In [7]:
with open("data/zone_30.geojson", "r") as f:
    geojson_data = json.load(f)

# Alle polygons inlezen als shapely geometrieën
zones = []
for feature in geojson_data["features"]:
    geom = shape(feature["geometry"])
    zones.append(geom)

## Verwerken en filteren van ongevallen

Filter eerst op Provincie Antwerpen en enkel voetgangers-gerelateerde gevallen.

In [8]:
ongevallen_data = ongevallen_df.filter((F.col("TX_PROV_COLLISION_NL") == "Provincie Antwerpen"))

ongevallen_data = ongevallen_data.filter(
    (F.col("TX_ROAD_USR_TYPE1_NL") == "Voetganger") 
    | (F.col("TX_ROAD_USR_TYPE2_NL") == "Voetganger")
)

# Filter op tijdstippen (ochtendspits en namiddagspits) — pas aan indien gewenst
ongevallen_data = ongevallen_data.filter((F.col("DT_TIME") == "08") | (F.col("DT_TIME") == "09")| (F.col("DT_TIME") == "15")| (F.col("DT_TIME") == "16")| (F.col("DT_TIME") == "17"))

# ongevallen_data.count()

## Afstand berekenen

Retourneert afstand in meters tussen twee latitude/longitude punten.

In [9]:
def haversine(lat1, lon1, lat2, lon2):
    """
        Berekenen van de afstand tussen 2 punten op de aarde
    """
    R = 6371000  # Earth radius in meters

    # convert degrees to radians
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])

    dlat = lat2 - lat1
    dlon = lon2 - lon1

    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.asin(math.sqrt(a))

    return R * c

## Data collecteren voor gebruik in de kaartcel

In [10]:
# Let op: .collect() haalt alle data naar de driver en is niet schaalbaar bij grote datasets.
# Dit is ok voor kleine subsets in een notebook, maar bij veel rijen graag alternatieve strategie gebruiken (bv. limit, sample, of schrijven naar file).
basisonderwijs = basisonderwijs_df.collect()
ongevallen = ongevallen_data.collect()

In [11]:
from ipywidgets import interact, IntSlider, Layout
# Slider maken voor leeftijdsgroepen
distance_slider = IntSlider(
    value=100,
    min=0,
    max=500,
    step=25,
    description='Afstand:',
    continuous_update=False,
    layout=Layout(width='500px')
)

@interact(distance_point=distance_slider)
def kaart(distance_point):
    m = folium.Map(location=[51.217098, 4.417978], zoom_start=13)

    # Maak twee lagen: één voor basisscholen en één voor ongevallen
    schools_layer = folium.FeatureGroup(name="Basisscholen", show=True)

    # Parent accidents layer zodat sublagen gegroepeerd worden in LayerControl
    accidents_parent = folium.FeatureGroup(name="Ongevallen", show=True)
    # Subgroepen binnen de Ongevallen parent laag
    accidents_in_30 = FeatureGroupSubGroup(accidents_parent, 'In 30-zone')
    accidents_out_30 = FeatureGroupSubGroup(accidents_parent, 'Buiten 30-zone')

    # Tekenen helper: accepteert een layer
    def drawCirkle(lat, lon, color, text="", layer=None):
        circle = folium.Circle(
            location=[lat, lon],
            radius=10,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.7,
            tooltip=text,
        )
        if layer is None:
            circle.add_to(m)
        else:
            circle.add_to(layer)

    folium.GeoJson(geojson_data, name="Zone 30").add_to(m)

    # Alle punten van ongevallen plotten
    for ongeval in ongevallen:
        if not ongeval["MS_X_COORD"]:
            continue

        lat, lon = lambert_to_latlon(
            transformer,
            float(ongeval["MS_X_COORD"].replace(',', '.')),
            float(ongeval["MS_Y_COORD"].replace(',', '.'))
        )

        # Check of het in de buurt van een school ligt
        inBerijk = False
        for school in basisonderwijs:
            lat_school, lon_school = lambert_to_latlon(
                transformer2,
                float(school['X'].replace(',', '.')),
                float(school['Y'].replace(',', '.')),
            )

            distance = haversine(lat, lon, lat_school, lon_school)
            if distance < distance_point:
                inBerijk = True
                continue

        if inBerijk:
            punt = Point(lon, lat)
            in_zone = any(punt.within(zone) for zone in zones)

            # Kies subgroup op basis van in_zone
            if in_zone:
                drawCirkle(lat, lon, "orange", "Weer: " +  ongeval["TX_WEATHER_NL"] + "<br>Weg conditie: " + ongeval["TX_ROAD_CONDITION_NL"] + "<br>Soort: " + ongeval["TX_CLASS_ACCIDENTS_NL"] + "<br>" + ongeval["TX_ROAD_USR_TYPE1_NL"] + "," + ongeval["TX_ROAD_USR_TYPE2_NL"], layer=accidents_in_30)
            else:
                drawCirkle(lat, lon, "red", "Weer: " +  ongeval["TX_WEATHER_NL"] + "<br>Weg conditie: " + ongeval["TX_ROAD_CONDITION_NL"] + "<br>Soort: " + ongeval["TX_CLASS_ACCIDENTS_NL"] + "<br>" + ongeval["TX_ROAD_USR_TYPE1_NL"] + "," + ongeval["TX_ROAD_USR_TYPE2_NL"], layer=accidents_out_30)

    # Basisscholen tekenen
    for school in basisonderwijs:
        lat, lon = lambert_to_latlon(
            transformer2,
            float(school['X'].replace(',', '.')),
            float(school['Y'].replace(',', '.')),
        )
        drawCirkle(lat, lon, "black", school["NAAM"], layer=schools_layer)

    # Voeg de parent en sublagen toe aan de kaart en voeg een LayerControl toe
    accidents_parent.add_to(m)
    accidents_in_30.add_to(m)
    accidents_out_30.add_to(m)
    schools_layer.add_to(m)

    # Zorg dat de LayerControl de subgroepen laat zien (collapsed=False helpt bij debug)
    folium.LayerControl(collapsed=False).add_to(m)

    m.save("tussentijdse_opdracht.html")

    display(m)


interactive(children=(IntSlider(value=100, continuous_update=False, description='Afstand:', layout=Layout(widt…

In [12]:
# m = folium.Map(location=[51.217098, 4.417978], zoom_start=13)

# # Maak twee lagen: één voor basisscholen en één voor ongevallen
# schools_layer = folium.FeatureGroup(name="Basisscholen", show=True)

# # Parent accidents layer zodat sublagen gegroepeerd worden in LayerControl
# accidents_parent = folium.FeatureGroup(name="Ongevallen", show=True)
# # Subgroepen binnen de Ongevallen parent laag
# accidents_in_30 = FeatureGroupSubGroup(accidents_parent, 'In 30-zone')
# accidents_out_30 = FeatureGroupSubGroup(accidents_parent, 'Buiten 30-zone')

# # Tekenen helper: accepteert een layer
# def drawCirkle(lat, lon, color, text="", layer=None):
#     circle = folium.Circle(
#         location=[lat, lon],
#         radius=10,
#         color=color,
#         fill=True,
#         fill_color=color,
#         fill_opacity=0.7,
#         tooltip=text,
#     )
#     if layer is None:
#         circle.add_to(m)
#     else:
#         circle.add_to(layer)

# folium.GeoJson(geojson_data, name="Zone 30").add_to(m)

# # Alle punten van ongevallen plotten
# for ongeval in ongevallen:
#     if not ongeval["MS_X_COORD"]:
#         continue

#     lat, lon = lambert_to_latlon(
#         transformer,
#         float(ongeval["MS_X_COORD"].replace(',', '.')),
#         float(ongeval["MS_Y_COORD"].replace(',', '.'))
#     )

#     # Check of het in de buurt van een school ligt
#     inBerijk = False
#     for school in basisonderwijs:
#         lat_school, lon_school = lambert_to_latlon(
#             transformer2,
#             float(school['X'].replace(',', '.')),
#             float(school['Y'].replace(',', '.')),
#         )

#         distance = haversine(lat, lon, lat_school, lon_school)
#         if distance < 100:
#             inBerijk = True
#             continue

#     if inBerijk:
#         punt = Point(lon, lat)
#         in_zone = any(punt.within(zone) for zone in zones)

#         # Kies subgroup op basis van in_zone
#         if in_zone:
#             drawCirkle(lat, lon, "orange", "Weer: " +  ongeval["TX_WEATHER_NL"] + ", Weg conditie: " + ongeval["TX_ROAD_CONDITION_NL"] + "Soort: " + ongeval["TX_CLASS_ACCIDENTS_NL"], layer=accidents_in_30)
#         else:
#             drawCirkle(lat, lon, "red", "Weer: " +  ongeval["TX_WEATHER_NL"] + ", Weg conditie: " + ongeval["TX_ROAD_CONDITION_NL"]+ "Soort: " + ongeval["TX_CLASS_ACCIDENTS_NL"], layer=accidents_out_30)

# # Basisscholen tekenen
# for school in basisonderwijs:
#     lat, lon = lambert_to_latlon(
#         transformer2,
#         float(school['X'].replace(',', '.')),
#         float(school['Y'].replace(',', '.')),
#     )
#     drawCirkle(lat, lon, "black", school["NAAM"], layer=schools_layer)

# # Voeg de parent en sublagen toe aan de kaart en voeg een LayerControl toe
# accidents_parent.add_to(m)
# accidents_in_30.add_to(m)
# accidents_out_30.add_to(m)
# schools_layer.add_to(m)

# # Zorg dat de LayerControl de subgroepen laat zien (collapsed=False helpt bij debug)
# folium.LayerControl(collapsed=False).add_to(m)

# m.save("tussentijdse_opdracht.html")

# m